In [1]:
import os 

os.chdir("../")
%pwd

'/home/leksman/Desktop/EEEEE/end_to_end_plant_vilage_project'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    dir_root: Path
    test_data_root: Path
    load_trained_model: Path
    num_epoch: int
    learning_rate: float
    num_classes: int
    batch_size: int
    num_workers: int
    shuffle: bool
    classification_report_loc: Path
    mlflow_url: str
  

In [3]:
from scr.Plant_Vilage.constants import SCHEMA_FILE_PATH,PARAMS_FILE_PATH,CONFIG_FILE_PATH
import scr.Plant_Vilage.utils.common as common
create_diretory = common.create_directory 
read_yaml = common.read_yaml


class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        schema_file_path: Path = SCHEMA_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH
    ):
       
        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(params_file_path)
        create_diretory([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.model_params
        create_diretory([config.dir_root])

        model_evaluation_config = ModelEvaluationConfig(
            dir_root= config.dir_root,
            load_trained_model=config.load_trained_model,
            num_epoch = params.num_epoch,
            learning_rate=params.learning_rate,
            num_classes=params.num_classes,
            batch_size=params.batch_size,
            num_workers=params.num_workers,
            shuffle=params.shuffle,
            classification_report_loc= config.classification_report_loc,
            test_data_root=config.test_data_root,
            mlflow_url=config.mlflow_url
            
    

        )

        return model_evaluation_config




In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import mlflow
from urllib.parse import urlparse
from sklearn.metrics import classification_report
import pandas as pd
import os
from scr.Plant_Vilage.components.model_trainer import get_model_optimizer


os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/leksman/end_to_end_plant_vilage_project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "leksman"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "38aba5d8d599d1c5648f9c3d7aa353c04ce2c9f9" 







class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def evaluate_model(self):
        # Transforms
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=15),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                 std=[0.5, 0.5, 0.5])
        ])

        # Dataset + loader
        test_dataset = datasets.ImageFolder(
            root=self.config.test_data_root,
            transform=transform
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=self.config.batch_size,
            shuffle=self.config.shuffle,
            num_workers=self.config.num_workers
        )

        # Load model
        model, optimizer, lossFun = get_model_optimizer()
        model.load_state_dict(torch.load(self.config.load_trained_model, map_location=self.device, weights_only=True))
        model.to(self.device)
    

        # MLflow setup
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type = urlparse(mlflow.get_tracking_uri()).scheme

        # Evaluation
        correct, total = 0, 0
        criterion = nn.CrossEntropyLoss()
        total_loss = 0.0
        all_labels, all_preds = [], []

        with mlflow.start_run():
            model.eval()
            with torch.no_grad():
                for X_val, y_val in test_loader:
                    X_val = X_val.to(self.device)
                    y_val = y_val.to(self.device).long()

                    # Forward
                    y_pred = model(X_val)

                    # Loss
                    loss = criterion(y_pred, y_val)
                    total_loss += loss.item()

                    # Predictions
                    pred_labels = y_pred.argmax(dim=1)
                    correct += (pred_labels == y_val).sum().item()
                    total += y_val.size(0)

                    # Collect for classification report
                    all_labels.extend(y_val.cpu().numpy())
                    all_preds.extend(pred_labels.cpu().numpy())

            # Metrics
            accuracy = correct / total
            avg_loss = total_loss / len(test_loader)

            # Log metrics
            mlflow.log_metric("test_accuracy", accuracy)
            mlflow.log_metric("test_loss", avg_loss)

            # Classification report
            report_dict = classification_report(
                all_labels,
                all_preds,
                target_names=test_dataset.classes,
                output_dict=True
            )

            # Save classification report as CSV
            report_df = pd.DataFrame(report_dict).transpose()
            report_csv_path =  self.config.classification_report_loc
            report_df.to_csv(report_csv_path, index=True)

            # Log report file to MLflow
            mlflow.log_artifact(report_csv_path)

            if tracking_url_type != "file":
                # Log model itself
                mlflow.pytorch.log_model(model, "cnn_model")
            else:
                mlflow.pytorch.log_model(model)

                 


[2025-09-16 02:40:15,534: INFO: utils: NumExpr defaulting to 16 threads.:]


In [5]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_eval = ModelEvaluation(config=model_evaluation_config)
    model_eval.evaluate_model()
except Exception as e:
    raise e    

[2025-09-16 02:40:17,207: INFO: common: YAML file loaded successfully from: config_yaml/config.yaml:]
[2025-09-16 02:40:17,210: INFO: common: YAML file loaded successfully from: schema.yaml:]
[2025-09-16 02:40:17,213: INFO: common: YAML file loaded successfully from: params.yaml:]
[2025-09-16 02:40:17,214: INFO: common: Created directory at: artifacts:]
[2025-09-16 02:40:17,215: INFO: common: Created directory at: artifacts/model_evaluation:]


2025/09/16 02:40:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run angry-shrew-612 at: https://dagshub.com/leksman/end_to_end_plant_vilage_project.mlflow/#/experiments/0/runs/ddee76f059634acc9bb5cdc6db91215a
🧪 View experiment at: https://dagshub.com/leksman/end_to_end_plant_vilage_project.mlflow/#/experiments/0
